# Seq2Seq
seq를 받아서 seq를 출력하는 모델
ex) Chatbot

Encoder-Decoder Architecture : RNN 두개를 이어붙인 형태

https://github.com/deeplearningzerotoall/PyTorch/blob/master/lab-11_5_seq2seq.ipynb


https://sanghyu.tistory.com/52 : RNN/LSTM/GRU

In [2]:
import random
import torch
import torch.nn as nn
from torch import optim

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [4]:
raw = ["I feel hungry.	나는 배가 고프다.",
       "Pytorch is very easy.	파이토치는 매우 쉽다.",
       "Pytorch is a framework for deep learning.	파이토치는 딥러닝을 위한 프레임워크이다.",
       "Pytorch is very clear to use.	파이토치는 사용하기 매우 직관적이다."]

In [5]:
SOS_token = 0
EOS_token = 1

In [6]:
class Vocab:
  def __init__(self):
    self.vocab2index = {"<SOS>":SOS_token, "<EOS>":EOS_token}
    self.index2vocab = {SOS_token: "<SOS>", EOS_token: "<EOS>"}
    self.vocab_count = {}
    self.n_vocab = len(self.vocab2index)
  
  def add_vocab(self, sentence):
    for word in sentence.split(" "):
      if word not in self.vocab2index:
        self.vocab2index[word] = self.n_vocab
        self.vocab_count[word] = 1
        self.index2vocab[self.n_vocab] = word
        self.n_vocab += 1
      else:
        self.vocab_count[word] += 1

In [7]:
def filter_pair(pair, source_max_length, target_max_length):
  return len(pair[0].split(" ")) < source_max_length and len(pair[1].split(" ")) < target_max_length

In [8]:
def preprocess(corpus, source_max_length, target_max_length):
  print("Reading corpus...")
  pairs = []
  for line in corpus:
    pairs.append([s for s in line.strip().lower().split("\t")])
  pairs = [pair for pair in pairs if filter_pair(pair, source_max_length, target_max_length)]

  source_vocab = Vocab()
  target_vocab = Vocab()

  for pair in pairs:
    source_vocab.add_vocab(pair[0])
    target_vocab.add_vocab(pair[1])
  print(f"Source vocab size = {source_vocab.n_vocab}")
  print(f"Target vocab size = {target_vocab.n_vocab}")
  
  return pairs, source_vocab, target_vocab

# Encoder AND Decoder

nn.Module.Embedding  
https://tutorials.pytorch.kr/beginner/nlp/word_embeddings_tutorial.html

In [9]:
class Encoder(nn.Module):
  def __init__(self, input_size, hidden_size):
    # input_size: load_source_vocab.n_vocab, hidden_size: encoder_hidden_size
    super(Encoder, self).__init__()
    self.hidden_size = hidden_size
    self.embedding = nn.Embedding(input_size, hidden_size)
    self.gru = nn.GRU(hidden_size, hidden_size)
  
  def forward(self, x, hidden=None):
    # x.shape:      (BatchSize, seq_length, input_size)
    # hidden.shape: ()
    x = self.embedding(x)
    if hidden is None:
      x, hidden = self.gru(x)
    else:
      x, hidden = self.gru(x, hidden)
    return x, hidden

class Decoder(nn.Module):
  def __init__(self, hidden_size, output_size):
    # hidden_size: 
    super(Decoder, self).__init__()
    self.hidden_size = hidden_size
    self.embedding = nn.Embedding(output_size, hidden_size)
    self.gru = nn.GRU(hidden_size, hidden_size)
    self.out = nn.Linear(hidden_size, output_size)
    self.softmax = nn.LogSoftmax(dim=1)

  def forward(self, x, hidden):
    x = self.embedding(x).view(1, 1, -1)
    x, hidden = self.gru(x, hidden)
    x = self.softmax(self.out(x[0]))
    return x, hidden  

In [10]:
embedder = nn.Embedding(10, 20)
input123 = torch.Tensor([1, 2, 3,9]).long()

In [11]:
input123.shape

torch.Size([4])

In [12]:
embedder(input123)

tensor([[ 2.2454,  0.6160,  0.3762,  1.1042, -0.6170, -1.7499,  0.1626,  0.0293,
         -0.3601, -1.6311,  1.1393,  1.7534,  0.7022,  0.2545,  1.8988,  0.4140,
          0.5596, -0.6938,  0.5742,  0.4515],
        [-0.6523,  1.5012, -0.1045,  0.6132,  0.0273,  0.8237, -1.0214, -1.2036,
          0.5162,  0.4059, -1.1705, -0.4971,  0.1649,  0.3223, -0.0993,  0.2233,
          0.1530, -0.5136,  0.4821, -1.4883],
        [ 0.0100,  0.6149, -2.2584, -2.2100, -1.1697, -1.6807,  1.6147, -0.8741,
         -1.4947,  0.6618,  1.0413, -0.0086, -0.7682,  0.4890,  0.2287, -0.5541,
          1.7901, -1.8499, -0.9343, -1.9813],
        [-0.6310, -0.1791, -0.6216, -0.6220,  1.4698, -2.8572,  1.1497,  1.0330,
         -0.7888,  1.0258, -1.4246,  2.0814,  0.6188,  0.8315,  0.4887,  0.4206,
         -0.4815, -0.3580, -0.3857,  0.3697]], grad_fn=<EmbeddingBackward0>)

In [13]:
def tensorize(vocab, sentence):
  indexes = [vocab.vocab2index[word] for word in sentence.split(" ")]
  indexes.append(vocab.vocab2index["<EOS>"])
  return torch.Tensor(indexes).long().to(device).view(-1, 1)

In [31]:
def train(pairs, source_vocab, target_vocab, encoder, decoder, n_iter, print_every=1000, learning_rate=0.01):
  loss_total = 0

  encoder_optimizer = optim.SGD(encoder.parameters(), lr=learning_rate)
  decoder_optimizer = optim.SGD(decoder.parameters(), lr=learning_rate)

  training_batch = [random.choice(pairs) for _ in range(n_iter)]
  training_source = [tensorize(source_vocab, pair[0]) for pair in training_batch]
  training_target = [tensorize(target_vocab, pair[1]) for pair in training_batch]

  criterion = nn.NLLLoss()

  for i in range(1, n_iter+1):
    source_tensor = training_source[i-1]
    target_tensor = training_target[i-1]
    
    encoder_hidden = torch.zeros([1, 1, encoder.hidden_size]).to(device)    

    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()

    source_length = source_tensor.size(0)
    target_length = target_tensor.size(0)

    loss = 0

    _, encoder_hidden = encoder(source_tensor)

    decoder_input = torch.Tensor([[SOS_token]]).long().to(device)
    decoder_hidden = encoder_hidden

    for di in range(target_length):
      decoder_output, decoder_hidden = decoder(decoder_input, decoder_hidden)
      loss += criterion(decoder_output, target_tensor[di])
      decoder_input = target_tensor[di]

    decoder_output, _ = decoder(decoder_input, decoder_hidden)

    loss.backward()

    encoder_optimizer.step()
    decoder_optimizer.step()

    loss_iter = loss.item() / target_length
    loss_total += loss_iter
    
    if i % print_every == 0:
      loss_avg = loss_total / print_every
      loss_total = 0
      print(f"[{i} - {i / n_iter * 100}] loss = {loss_avg:05.4f}")

In [40]:
def evaluate(pairs, source_vocab, target_vocab, encoder, decoder, target_max_length):
  for pair in pairs:
    print(">", pair[0])
    print("=", pair[1])
    source_tensor = tensorize(source_vocab, pair[0])
    source_length = source_tensor.size()[0]

    _, encoder_hidden = encoder(source_tensor)
    # encoder_hidden : (D * layers, N, H_out)
    decoder_input = torch.Tensor([[SOS_token]], device=device).long()
    decoder_hidden = encoder_hidden
    decoded_words = []

    for di in range(target_max_length):
      decoder_output, decoder_hidden = decoder(decoder_input, decoder_hidden)
      _, top_index = decoder_output.data.topk(1)
      if top_index.item() == EOS_token:
        decoded_words.append("<EOS>")
        break
      else:
        decoded_words.append(target_vocab.index2vocab[top_index.item()])
      decoder_input = top_index.squeeze()
    
    predict_words = decoded_words
    predict_sentence = " ".join(predict_words)
    print("<", predict_sentence)
    print("")

# Main process

In [43]:
SOURCE_MAX_LENGTH = 10
TARGET_MAX_LENGTH = 12

In [44]:
load_pairs, load_source_vocab, load_target_vocab = preprocess(raw, SOURCE_MAX_LENGTH, TARGET_MAX_LENGTH)
print(load_pairs)

Reading corpus...
Source vocab size = 17
Target vocab size = 13
[['i feel hungry.', '나는 배가 고프다.'], ['pytorch is very easy.', '파이토치는 매우 쉽다.'], ['pytorch is a framework for deep learning.', '파이토치는 딥러닝을 위한 프레임워크이다.'], ['pytorch is very clear to use.', '파이토치는 사용하기 매우 직관적이다.']]


In [45]:
encoder_hidden_size = 16
decoder_hidden_size = encoder_hidden_size

encoder = Encoder(load_source_vocab.n_vocab, encoder_hidden_size).to(device)
decoder = Decoder(decoder_hidden_size, load_target_vocab.n_vocab).to(device)

In [46]:
train(load_pairs, load_source_vocab, load_target_vocab, encoder, decoder, 5000, print_every=1000)

[1000 - 20.0] loss = 0.7020
[2000 - 40.0] loss = 0.0997
[3000 - 60.0] loss = 0.0315
[4000 - 80.0] loss = 0.0163
[5000 - 100.0] loss = 0.0110


In [47]:
evaluate(load_pairs, load_source_vocab, load_target_vocab, encoder, decoder, TARGET_MAX_LENGTH)

> i feel hungry.
= 나는 배가 고프다.
< 나는 배가 고프다. <EOS>

> pytorch is very easy.
= 파이토치는 매우 쉽다.
< 파이토치는 매우 쉽다. <EOS>

> pytorch is a framework for deep learning.
= 파이토치는 딥러닝을 위한 프레임워크이다.
< 파이토치는 딥러닝을 위한 프레임워크이다. <EOS>

> pytorch is very clear to use.
= 파이토치는 사용하기 매우 직관적이다.
< 파이토치는 사용하기 매우 직관적이다. <EOS>

